In [1]:
import torch
import numpy as np
import torch.nn as nn
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import torch.nn.functional as F

torch over numpy to run on gpu-s for faster processing

we will use nn.linear class to initialize weights and biases of a linear model

use tensordataset to split the data into smaller chunks to run code, returns a tuple with 2 elements, in our cse inputs and targets

use dataloader to split data into bathces of predefined size; enbles shuffling and ramdom sampling of data

use nn.functional to utilize its mse_loss for the loss function

In [2]:
print(torch.__version__)

2.1.2


In [3]:
t1=torch.tensor(4.0)
t1

tensor(4.)

In [4]:
t1.dtype

torch.float32

In [5]:
t2=torch.tensor([1.,2,3,4])
t2

tensor([1., 2., 3., 4.])

In [6]:
t2.dtype

torch.float32

In [7]:
t3=torch.tensor([[5.,6],
                 [7,8],
                 [9,10]])
t3

tensor([[ 5.,  6.],
        [ 7.,  8.],
        [ 9., 10.]])

In [8]:
t4=torch.tensor([
    [[11,12,13],
     [13,14,15]],
     [[15,16,17],
      [17,18,19]]
])
t4

tensor([[[11, 12, 13],
         [13, 14, 15]],

        [[15, 16, 17],
         [17, 18, 19]]])

In [9]:
print(t1)
t1.shape

tensor(4.)


torch.Size([])

In [10]:
print(t2)
t2.shape

tensor([1., 2., 3., 4.])


torch.Size([4])

In [11]:
print(t3)
t3.shape

tensor([[ 5.,  6.],
        [ 7.,  8.],
        [ 9., 10.]])


torch.Size([3, 2])

In [12]:
print(t4)
t4.shape

tensor([[[11, 12, 13],
         [13, 14, 15]],

        [[15, 16, 17],
         [17, 18, 19]]])


torch.Size([2, 2, 3])

In [13]:
# bnasic tensor operations
x=torch.tensor(3.0)
w=torch.tensor(4.,requires_grad=True)
b=torch.tensor(5.,requires_grad=True)
x,w,b

(tensor(3.), tensor(4., requires_grad=True), tensor(5., requires_grad=True))

In [14]:
x.is_leaf

True

In [15]:
y=w*x+b
y

tensor(17., grad_fn=<AddBackward0>)

In [16]:
# y.retain_grad()
y.backward()
print('dy/dx:',x.grad)
print('dy/dw:',w.grad)
print('dy/db:',b.grad)
print('dy/dw 2:',w.grad_fn)

dy/dx: None
dy/dw: tensor(3.)
dy/db: tensor(1.)
dy/dw 2: None


In [17]:
w2=torch.tensor(42.,requires_grad=True)
b2=torch.tensor(52.,requires_grad=True)
y2=w2+b2

In [18]:
y2.backward
print(w2.grad)

None


In [19]:
t6=torch.full((3,2),42)
t6

tensor([[42, 42],
        [42, 42],
        [42, 42]])

In [20]:
t7=torch.concatenate((t3,t6))
t7

tensor([[ 5.,  6.],
        [ 7.,  8.],
        [ 9., 10.],
        [42., 42.],
        [42., 42.],
        [42., 42.]])

In [21]:
t8=torch.sin(t7)
t8

tensor([[-0.9589, -0.2794],
        [ 0.6570,  0.9894],
        [ 0.4121, -0.5440],
        [-0.9165, -0.9165],
        [-0.9165, -0.9165],
        [-0.9165, -0.9165]])

In [22]:
t9=t8.reshape(3,2,2)
t9

tensor([[[-0.9589, -0.2794],
         [ 0.6570,  0.9894]],

        [[ 0.4121, -0.5440],
         [-0.9165, -0.9165]],

        [[-0.9165, -0.9165],
         [-0.9165, -0.9165]]])

In [23]:
x=np.array([[1,2],
            [3,4.]])
x

array([[1., 2.],
       [3., 4.]])

In [24]:
y=torch.from_numpy(x)
y

tensor([[1., 2.],
        [3., 4.]], dtype=torch.float64)

In [25]:
x.dtype,y.dtype

(dtype('float64'), torch.float64)

In [26]:
z=y.numpy()
z

array([[1., 2.],
       [3., 4.]])

Fundamentals of Pytorch and Numpy done.
Next progress towards linear regression znd gradient descent

Video course Timestamp 25.28 / 1.43.32  

In [27]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

In [28]:
# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

In [29]:
# Convert inputs and targets to tensors
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [30]:
# Weights and biases
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w)
print(b)

tensor([[ 1.5357, -0.7685, -1.5126],
        [ 1.0324,  0.2342, -0.2568]], requires_grad=True)
tensor([ 0.7798, -2.1890], requires_grad=True)


In [31]:
inputs @ w.t()+b

tensor([[ -3.6413,  77.8271],
        [-23.9004,  95.9359],
        [-56.3170, 104.1209],
        [ 68.4130, 103.6875],
        [-72.9093,  73.5550]], grad_fn=<AddBackward0>)

In [32]:
def model(x):
    return x @ w.t()+b

In [33]:
preds=model(inputs)
print(preds)

tensor([[ -3.6413,  77.8271],
        [-23.9004,  95.9359],
        [-56.3170, 104.1209],
        [ 68.4130, 103.6875],
        [-72.9093,  73.5550]], grad_fn=<AddBackward0>)


In [34]:
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [35]:
diff=preds-targets
torch.sum(diff * diff)/diff.numel()

tensor(8582.8867, grad_fn=<DivBackward0>)

to get rid of negatives in diff, square numbers; * does a element wise multiplication

In [36]:
# mean squared error loss 
def mse(t1,t2):
    diff=t1-t2
    return torch.sum(diff*diff)/diff.numel()

In [37]:
loss=mse(preds,targets)
print(loss)

tensor(8582.8867, grad_fn=<DivBackward0>)


In [38]:
loss.backward()

In [39]:
print(w)
print(w.grad)

tensor([[ 1.5357, -0.7685, -1.5126],
        [ 1.0324,  0.2342, -0.2568]], requires_grad=True)
tensor([[ -7311.1890, -10322.2432,  -6008.5918],
        [   252.8978,  -1057.2351,   -475.2467]])


In [40]:
print(b)
print(b.grad)

tensor([ 0.7798, -2.1890], requires_grad=True)
tensor([-93.8710,  -0.9747])


In [41]:
with torch.no_grad():
    w-=w.grad*1e-5
    b-=b.grad*1e-5

In [42]:
w,b

(tensor([[ 1.6088, -0.6652, -1.4525],
         [ 1.0299,  0.2448, -0.2521]], requires_grad=True),
 tensor([ 0.7808, -2.1890], requires_grad=True))

In [43]:
preds=model(inputs)
loss=mse(preds,targets)
print(loss)

tensor(6781.2764, grad_fn=<DivBackward0>)


the grad values for w and b should be reset to zero, else the next operation will add to existing grad values

In [44]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


to train the model repeat the above operation x number of times, called epochs

In [45]:
for i in range(100):
    preds=model(inputs)
    loss=mse(preds,targets)
    loss.backward()
    with torch.no_grad():
        w-=w.grad*1e-5
        b-=b.grad*1e-5
        w.grad.zero_()
        b.grad.zero_()

In [46]:
preds=model(inputs)
loss=mse(preds,targets)
print(loss)

tensor(1001.1504, grad_fn=<DivBackward0>)


In [47]:
preds

tensor([[ 69.7998,  76.8088],
        [ 79.3334,  98.3465],
        [104.9187, 127.7869],
        [ 92.6086,  76.2472],
        [ 55.3346,  91.7690]], grad_fn=<AddBackward0>)

In [48]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

xxxxxxxxx
Starting work on using Troch for gradient descent

In [49]:
# inputs are temperature, rainfall, humidity
inputs=np.array([[73,67,43],
                 [91,88,64],
                 [87,134,58],
                 [102,43,37],
                 [69,96,70],
                 [74,66,43],
                 [91,87,65],
                 [88,114,59],
                 [101,44,37],
                 [68,96,71],
                 [73,66,44],
                 [92,87,64],
                 [87,135,57],
                 [103,43,36],
                 [68,97,70]],
                 dtype='float')

# targets are apples, oranges
targets=np.array([[56,70],
                  [81,101],
                  [119,133],
                  [22,37],
                  [103,119],
                  [57,69],
                  [80,102],
                  [118,132],
                  [21,38],
                  [104,119],
                  [57,69],
                  [82,100],
                  [118,134],
                  [20,38],
                  [102,120]],
                  dtype='float')

inputs=torch.from_numpy(inputs)
targets=torch.from_numpy(targets)

In [50]:
inputs.shape

torch.Size([15, 3])

In [51]:
targets.shape

torch.Size([15, 2])

use tensordataset to split the data into smaller chunks to run code, returns a tuple with 2 elements, in our cse inputs and targets

In [52]:
train_ds=TensorDataset(inputs,targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]], dtype=torch.float64),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]], dtype=torch.float64))

use dataloader to split data into bathces of predefined size; enbles shuffling and ramdom sampling of data

In [53]:
batch_size=5
train_dl=DataLoader(train_ds,batch_size,shuffle=True)

In [54]:
for xb,yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[ 68.,  97.,  70.],
        [ 68.,  96.,  71.],
        [ 73.,  67.,  43.],
        [103.,  43.,  36.],
        [ 73.,  66.,  44.]], dtype=torch.float64)
tensor([[102., 120.],
        [104., 119.],
        [ 56.,  70.],
        [ 20.,  38.],
        [ 57.,  69.]], dtype=torch.float64)


we will use nn.linear class to initialize weights and biases of a linear model

In [55]:
# defining model usin nn.linear
model=nn.Linear(3,2,dtype=float)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.4255,  0.1574, -0.3861],
        [-0.5232,  0.1012, -0.4477]], dtype=torch.float64, requires_grad=True)
Parameter containing:
tensor([0.4477, 0.1708], dtype=torch.float64, requires_grad=True)


In [56]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.4255,  0.1574, -0.3861],
         [-0.5232,  0.1012, -0.4477]], dtype=torch.float64, requires_grad=True),
 Parameter containing:
 tensor([0.4477, 0.1708], dtype=torch.float64, requires_grad=True)]

In [57]:
# generate predictions using the nn.linear model above
# inputs
preds=model(inputs)
preds

tensor([[-36.6741, -50.4941],
        [-49.1369, -67.1885],
        [-37.8777, -57.7553],
        [-50.4758, -65.4085],
        [-40.8324, -57.5553],
        [-37.2570, -51.1185],
        [-49.6804, -67.7374],
        [-41.8373, -60.7498],
        [-49.8929, -64.7842],
        [-40.7929, -57.4798],
        [-37.2176, -51.0430],
        [-49.7198, -67.8129],
        [-37.3342, -57.2064],
        [-50.5152, -65.4840],
        [-40.2494, -56.9309]], dtype=torch.float64, grad_fn=<AddmmBackward0>)

In [58]:
loss_fn=F.mse_loss

In [59]:
loss=loss_fn(model(inputs),targets)
print(loss)

tensor(19765.9237, dtype=torch.float64, grad_fn=<MseLossBackward0>)


use optimizer SGD, stochastic gradient descent, and a learning rate of 1e-5

In [60]:
opt=torch.optim.SGD(model.parameters(),lr=1e-5)

the function to train the model

In [66]:
# function to train the model
def fit(num_epochs,model,loss_fn,opt,train_dl):
    # repeat for x number of epochs
    for epoch in range(num_epochs):

        for xb,yb in train_dl:

            # step 1 generate predictions
            pred=model(xb)

            # step 2 calculate loss
            loss=loss_fn(pred,yb)

            # step 3 compute gradients
            loss.backward()

            # step 4 update parameters using gradients
            opt.step()

            # step 5 reset the gradients to zero
            opt.zero_grad()

        # print 
        if (epoch+1)%10==0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1,num_epochs,loss.item()))



In [71]:
fit(100,model,loss_fn,opt,train_dl)

Epoch [10/100], Loss: 2.2892
Epoch [20/100], Loss: 9.1643
Epoch [30/100], Loss: 2.3223
Epoch [40/100], Loss: 1.2108
Epoch [50/100], Loss: 16.8347
Epoch [60/100], Loss: 9.6417
Epoch [70/100], Loss: 7.6652
Epoch [80/100], Loss: 33.1145
Epoch [90/100], Loss: 37.1265
Epoch [100/100], Loss: 7.3980


In [72]:
print(w,b)

tensor([[ 1.0230,  0.5219, -0.9448],
        [ 0.5186,  0.6731, -0.0920]], requires_grad=True) tensor([ 0.7793, -2.1920], requires_grad=True)


In [73]:
preds=model(inputs)
preds

tensor([[ 58.0216,  71.4330],
        [ 81.9151, 100.3240],
        [123.5858, 138.5676],
        [ 21.0833,  38.0022],
        [101.0950, 117.5148],
        [ 56.6947,  70.2701],
        [ 81.5013, 100.1252],
        [104.9049, 121.0063],
        [ 22.4102,  39.1651],
        [102.0081, 118.4789],
        [ 57.6078,  71.2342],
        [ 80.5882,  99.1611],
        [123.9996, 138.7664],
        [ 20.1702,  37.0381],
        [102.4219, 118.6777]], dtype=torch.float64, grad_fn=<AddmmBackward0>)

In [74]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 119.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]], dtype=torch.float64)

lets test it for a new region

In [76]:
model(torch.tensor([[75,63,44.]],dtype=float))

tensor([[54.0129, 68.0084]], dtype=torch.float64, grad_fn=<AddmmBackward0>)